In [6]:
#@title Install required packages
!uv pip install -U pse # proxy structuring engine
!uv pip install sentencepiece
!uv pip install accelerate
!uv pip install transformers
!uv pip install torch
!uv pip install numpy
!uv pip install bitsandbytes
!uv pip install sentencepiece
!uv pip install protobuf
!uv pip install -U tqdm
!uv pip install ipywidgets

Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Resolved 26 packages in 8ms                                          
Building pse @ file:///Users/jckwind/Documents/proxy-structuring-engine
⠙ Preparing packages... (0/1)                                                   

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Building pse @ file:///Users/jckwind/Documents/proxy-structuring-engine
   Built pse @ file:///Users/jckwind/Documents/proxy-structuring-engine
Prepared 1 package in 341ms                                              
Uninstalled 1 package in 5ms
Installed 1 package in 1msle:///Users/jckwind/Documents/prox
 - pse==2.1.0
 + pse==2.1.0 (from file:///Users/jckwind/Documents/proxy-structuring-engine)
Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Audited 1 package in 1ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Audited 1 package in 2ms


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Resolved 1 package in 31ms                                           
Audited 1 package in 0.10ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Audited 1 package in 2ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
# @title Setup Llama 3.2 1B
import torch
from transformers import AutoTokenizer, LlamaForCausalLM

from pse.engine.structuring_engine import StructuringEngine
from pse.util.torch_mixin import PSETorchMixin


class PSE_Torch(PSETorchMixin, LlamaForCausalLM):
    pass


model_path = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = PSE_Torch.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)

model.config.pad_token_id = model.config.eos_token_id[0]
if model.generation_config:
    model.generation_config.pad_token_id = model.config.eos_token_id[0]

In [8]:

#@title Create engine and test json generation
import json

SIMPLE_JSON_SCHEMA = {
    "type": "object",
    "properties": {"value": {"type": "number"}},
    "required": ["value"],
}
model.engine = StructuringEngine(tokenizer)
model.engine.configure(SIMPLE_JSON_SCHEMA)
prompt = "Please generate a json object with the value 9.11, with the following schema:\n"
prompt += json.dumps(SIMPLE_JSON_SCHEMA, indent=2)
messages = [{"role": "user", "content": prompt}]
input_ids = tokenizer.apply_chat_template(
    messages, return_tensors="pt", add_generation_prompt=True
)
assert isinstance(input_ids, torch.Tensor)
input_ids = input_ids.to(model.device)
assert isinstance(input_ids, torch.Tensor)
output = model.generate(
    input_ids,
    do_sample=True,
    temperature=1.0,
)
print("Output:\n" + 100 * "-")
print(tokenizer.decode(output[0]))

Output:
----------------------------------------------------------------------------------------------------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 12 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Please generate a json object with the value 9.11, with the following schema:
{
  "type": "object",
  "properties": {
    "value": {
      "type": "number"
    }
  },
  "required": [
    "value"
  ]
}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{ "value": 9.11 }


In [9]:
# @title Test advanced-json generation
ADVANCED_JSON_SCHEMA = {
    "type": "object",
    "properties": {
        "name": {"const": "metacognition"},
        "arguments": {
            "type": "object",
            "properties": {
                "chain_of_thought": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": (
                        "A sequence of high-level thoughts, reasoning and internal dialogue.\n"
                        "Supports all Unicode characters, including emojis."
                    ),
                },
                "internal_state": {
                    "type": "string",
                    "description": (
                        "A description of the agent's internal state.\n"
                        "Supports all Unicode characters, including emojis."
                    ),
                    "nullable": True,
                },
            },
            "required": ["chain_of_thought"],
        },
    },
    "required": ["name", "arguments"],
}
model.engine.configure(ADVANCED_JSON_SCHEMA)
prompt = "This is a test. Use this schema to structure your response: {schema}"
prompt = prompt.format(schema=json.dumps(ADVANCED_JSON_SCHEMA, indent=2))

messages = [{"role": "user", "content": prompt}]
input_ids = tokenizer.apply_chat_template(
    messages, return_tensors="pt", add_generation_prompt=True
)
assert isinstance(input_ids, torch.Tensor)
input_ids = input_ids.to(model.device)
assert isinstance(input_ids, torch.Tensor)
greedy_output = model.generate(
    input_ids,
    do_sample=True,
    temperature=1.0,
    min_p=0.08,  # you can use your favorite sampling options!
)
print("Output:\n" + 100 * "-")
print(tokenizer.decode(greedy_output[0]))


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0